# Add PUDL IDs to small gens

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard libraries
import logging
import sys
import os
import pathlib

# 3rd party libraries
import geopandas as gpd
import dask.dataframe as dd
from dask.distributed import Client
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa

# Local libraries
import pudl

In [3]:
# Enable viewing of logging outputs
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [4]:
# Display settings
sns.set()
%matplotlib inline
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [5]:
frequency = 'AS' # annual

# Establish connection to pudl database
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine, freq=frequency) #annual frequency

In [6]:
# Access a table by name as an object
ferc_small = pudl_out.plants_small_ferc1()
ferc_big = pudl_out.plants_steam_ferc1()
eia = pudl_out.gens_eia860()

In [7]:
# Replicate PUDL ID mapping process
aa = pudl.glue.ferc1_eia.glue(ferc1=True, eia=True)
glue_tab = aa['plants_pudl']
glue_tab['plant_name_lower'] = glue_tab.plant_name_pudl.str.lower()
glue_tab.query("plant_name_pudl == 'Harris Lake'")

ferc_glue_mer = pd.merge(ferc_small, glue_tab, left_on='plant_name_original', right_on='plant_name_lower', how='left')
ferc_glue_mer['dup'] = ferc_glue_mer.duplicated(subset=['plant_name_original'], keep=False)

print(len(ferc_glue_mer[ferc_glue_mer['dup']]))
print(len(ferc_glue_mer))
print(len(ferc_small))

In [11]:
# preliminary look at how many of the plants that have been mapped also have eia technology descriptions

has_tech = eia[eia['technology_description'].notna()].copy()
print(len(ferc_glue_mer.plant_name_pudl.unique()))
print(len([x for x in ferc_glue_mer.plant_id_pudl.unique() if x in has_tech.plant_id_pudl.unique()]))
#print(len([x for x in matching_utility.plant_id_pudl_ferc if x in has_tech.plant_id_pudl]))

# Remember to only use plant id pudls that have one tech per plant id pudl :) 

1077
228


In [14]:
mapping = pd.read_excel('/Users/aesharpe/Desktop/mapping_eia923_ferc1_copy().xlsx', sheet_name='plants_combined')

eia_map = (
    mapping[mapping['plant_name_eia'].notna()]
    [['plant_name_pudl', 'plant_id_pudl', 'utility_name_eia', 'plant_name_eia']].copy())

unmapped_eia = (
    mapping[(mapping['plant_name_eia'].notna()) & (mapping['plant_name_ferc1'].isna())]
    [['plant_name_pudl', 'plant_id_pudl', 'utility_name_eia', 'plant_name_eia']].copy()
    .rename(columns={'plant_id_pudl': 'plant_id_pudl_eia'}))
unmapped_ferc = (
    mapping[(mapping['plant_name_ferc1'].notna()) & (mapping['plant_name_eia'].isna())]
    [['plant_name_pudl', 'plant_id_pudl', 'utility_name_ferc1', 'plant_name_ferc1']].copy()
    .rename(columns={'plant_id_pudl': 'plant_id_pudl_ferc'}))

In [15]:
mer2 = pd.merge(unmapped_eia, unmapped_ferc, on='plant_name_pudl', how='inner')

print(len(unmapped_eia))
print(len(unmapped_ferc))
print(len(mer2))

In [18]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def get_fuzzy_wit_it(small_ferc, mapping):
    small_ferc = small_ferc.dropna(subset=['plant_name_ferc1'])
    mapping = mapping.dropna(subset=['plant_name_eia'])
    
    small_ferc['output'] = (
        small_ferc.plant_name_ferc1.apply(lambda x: process.extract(x, mapping.plant_name_eia, limit=1, scorer=fuzz.token_sort_ratio)))
    small_ferc['output'] = (
        small_ferc.output.apply(pd.Series)[0])
    small_ferc['match'], small_ferc['ratio'], small_ferc['other'] = zip(*small_ferc.output)
    
    return small_ferc

/Users/aesharpe/miniconda3/envs/pudl-dev/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [32]:
bing = get_fuzzy_wit_it(dd, unmapped_eia)

In [31]:
zing.sort_values('ratio', ascending=False).to_excel('/Users/aesharpe/Desktop/matching_eia.xlsx')

In [34]:
bing.sort_values('ratio', ascending=False).to_excel('/Users/aesharpe/Desktop/matching_eia.xlsx')

In [19]:
mer3 = mer2.dropna(subset=['utility_name_eia', 'utility_name_ferc1']).copy()
mer3['ratio']= mer3.apply(lambda x: fuzz.partial_ratio(x['utility_name_ferc1'],x['utility_name_eia']),axis=1)
matching_utility = mer3[(mer3['ratio'] > 70) & (mer3['plant_id_pudl_eia'] != mer3['plant_id_pudl_ferc'])].copy()

In [40]:
# Used these as the first pudl ids to fix
len(matching_utility)

1

In [20]:
mer4 = mer3[mer3['plant_id_pudl_eia'] != mer3['plant_id_pudl_ferc']]

In [21]:
dd = ferc_small.drop_duplicates('plant_name_original')

In [22]:
len(dd.sort_values('plant_name_original'))

1930

In [54]:
dd.sort_values('plant_name_original').to_excel('/Users/aesharpe/Desktop/small-gen-to-map.xlsx')

In [82]:
mapping['plant_name_ferc1'] = mapping['plant_name_ferc1'].str.lower()
test = mapping[mapping['plant_name_ferc1'].isin(dd.plant_name_ferc1.unique().tolist())]
test[test['plant_id_eia'].isna()].sort_values('plant_name_ferc1')

,changes,plant_id_pudl,Unnamed: 2,plant_name_pudl,utility_id_ferc1,utility_name_ferc1,plant_name_ferc1,plant_id_eia,plant_name_eia,utility_name_eia,utility_id_eia,Notes
10253,NaN,9105,NaN,(Fish Water Unit),161.0,Southern California Edison Company,(fish water unit),NaN,NaN,NaN,NaN,"Capacity < 5MW, not mapped"
13922,NaN,11796,NaN,(hydro) project #539,82.0,kentucky utilities company,(hydro) project #539,NaN,NaN,NaN,NaN,2018 Addition
13375,NaN,11658,NaN,(icu),39.0,"connecticut light and power company, the",(icu),NaN,NaN,NaN,NaN,2018 Addition
13376,NaN,11659,NaN,(steam),39.0,"connecticut light and power company, the",(steam),NaN,NaN,NaN,NaN,2018 Addition
1439,NaN,1061,NaN,* Beacon,44.0,The Detroit Edison Company,* beacon,NaN,NaN,NaN,NaN,No EIA ID
...,...,...,...,...,...,...,...,...,...,...,...,...
10118,NaN,9014,NaN,Xerox Corp,141.0,Portland General Electric Company,xerox corp,NaN,NaN,NaN,NaN,"Capacity < 5MW, not mapped"
9574,NaN,8590,NaN,Yankee Solar #1,42.0,The Dayton Power and Light Company,yankee solar #1,NaN,NaN,NaN,NaN,"Capacity < 5MW, not mapped"
15955,NaN,12331,NaN,youngs corner,11.0,emera maine,youngs corner,NaN,NaN,NaN,NaN,2018 Addition - New 2018
11243,NaN,9952,NaN,Ypsilanti,44.0,The Detroit Edison Company,ypsilanti,NaN,NaN,NaN,NaN,"Capacity < 5MW, not mapped"


In [98]:
one = mapping.sort_values('plant_name_ferc1')[0:20].copy()
two = dd.sort_values('plant_name_ferc1')[0:10].copy()

In [100]:
get_fuzzy_wit_it(two, one)

TypeError: expected string or bytes-like object